In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install transformers
!pip install torch
!pip3 install deeppavlov
import numpy as np
from sklearn.metrics import precision_recall_fscore_support

from torch.utils.data import Dataset, DataLoader
from transformers import AdamW, get_linear_schedule_with_warmup

     |████████████████████████████████| 3.5 MB 34.1 MB/s 
     |████████████████████████████████| 67 kB 6.4 MB/s 
     |████████████████████████████████| 6.8 MB 66.9 MB/s 
     |████████████████████████████████| 596 kB 60.8 MB/s 
     |████████████████████████████████| 895 kB 54.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 880 kB 26.8 MB/s 
     |████████████████████████████████| 53 kB 3.1 MB/s 
     |████████████████████████████████| 40 kB 28 kB/s 
     |████████████████████████████████| 76 kB 6.3 MB/s 
     |████████████████████████████████| 20.1 MB 1.3 MB/s 
     |████████████████████████████████| 654 kB 66.6 MB/s 
     |████████████████████████████████| 859 kB 61.3 MB/s 
     |████████████████████████████████| 65 kB 5.3 MB/s 
     |████████████████████████████████| 1.5 MB 63.8 MB/s 
     |████████████████████████████████| 43 kB 2.3 

In [3]:
!cp  drive/MyDrive/НИР/dataset_roberta.py .
!cp  drive/MyDrive/НИР/functions_for_roberta.py .
from dataset_roberta import CustomDataset
from functions_for_roberta import Classifier

In [4]:
from torch import cuda

device = 'cuda' if cuda.is_available() else 'cpu'


In [5]:
"""from torch.utils.data import Dataset
import torch

class CustomDataset(Dataset):

  def __init__(self, texts, targets, tokenizer, max_len=512):
    self.texts = texts
    self.targets = targets
    self.tokenizer = tokenizer
    self.max_len = max_len

  def __len__(self):
    return len(self.texts)

  def __getitem__(self, idx):
    text = str(self.texts[idx])
    target = int(self.targets[idx])

    encoding = tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        max_length=512,
        return_token_type_ids=False,
        padding='max_length',
        return_attention_mask=True,
        pad_to_max_length=True,
        truncation=True 
    )

    return {
      'text': text,
      'input_ids': torch.tensor(encoding['input_ids']),
      'targets': torch.tensor(target, dtype=torch.long),
      'attention_mask': torch.tensor(encoding['attention_mask'])

      
    }"""

"from torch.utils.data import Dataset\nimport torch\n\nclass CustomDataset(Dataset):\n\n  def __init__(self, texts, targets, tokenizer, max_len=512):\n    self.texts = texts\n    self.targets = targets\n    self.tokenizer = tokenizer\n    self.max_len = max_len\n\n  def __len__(self):\n    return len(self.texts)\n\n  def __getitem__(self, idx):\n    text = str(self.texts[idx])\n    target = int(self.targets[idx])\n\n    encoding = tokenizer.encode_plus(\n        text,\n        add_special_tokens=True,\n        max_length=512,\n        return_token_type_ids=False,\n        padding='max_length',\n        return_attention_mask=True,\n        pad_to_max_length=True,\n        truncation=True \n    )\n\n    return {\n      'text': text,\n      'input_ids': torch.tensor(encoding['input_ids']),\n      'targets': torch.tensor(target, dtype=torch.long),\n      'attention_mask': torch.tensor(encoding['attention_mask'])\n\n      \n    }"

In [6]:
from transformers import AutoModel,  AutoConfig, AutoTokenizer

config = AutoConfig.from_pretrained("blinoff/roberta-base-russian-v0")
tokenizer = AutoTokenizer.from_pretrained("blinoff/roberta-base-russian-v0")
model = AutoModel.from_pretrained("blinoff/roberta-base-russian-v0")

/usr/local/lib/python3.7/dist-packages/cryptography/hazmat/backends/openssl/x509.py:17: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  utils.DeprecatedIn35,


Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/16.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.61M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.28M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/477M [00:00<?, ?B/s]

Some weights of the model checkpoint at blinoff/roberta-base-russian-v0 were not used when initializing RobertaModel: ['lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.decoder.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [7]:
import torch

out_features = model.encoder.layer[1].output.dense.out_features
#model.classifier = torch.nn.Linear(out_features, 2)

In [8]:
classifier_RoB = Classifier(
    model,
    tokenizer,
    out_features,
    1
)

In [9]:
from deeppavlov.dataset_readers.basic_classification_reader import BasicClassificationDatasetReader
from deeppavlov.dataset_iterators.basic_classification_iterator import BasicClassificationDatasetIterator

# read data from particular columns of `.csv` file
dr = BasicClassificationDatasetReader().read(
    data_path='/content/drive/MyDrive/НИР',
    train='Join_dataset.csv',
    x = 'text',
    y = 'news'
)

# initialize data iterator splitting `train` field to `train` and `valid` in proportion 0.8/0.2
train_iterator = BasicClassificationDatasetIterator(
    data=dr,
    field_to_split='train',  # field that will be splitted
    split_fields=['train', 'valid', 'test'],   # fields to which the fiald above will be splitted
    split_proportions=[0.5, 0.1, 0.4],  #proportions for splitting
    split_seed=23,  # seed for splitting dataset
    seed=42)  # seed for iteration over dataset

2022-02-10 20:11:05.734 WARNING in 'deeppavlov.dataset_readers.basic_classification_reader'['basic_classification_reader'] at line 112: Cannot find /content/drive/MyDrive/НИР/valid.csv file
2022-02-10 20:11:05.739 WARNING in 'deeppavlov.dataset_readers.basic_classification_reader'['basic_classification_reader'] at line 112: Cannot find /content/drive/MyDrive/НИР/test.csv file
2022-02-10 20:11:05.743 INFO in 'deeppavlov.dataset_iterators.basic_classification_iterator'['basic_classification_iterator'] at line 74: Splitting field <<train>> to new fields <<['train', 'valid', 'test']>>


In [10]:
x_train, y_train = train_iterator.get_instances(data_type='train')

x_valid, y_valid = train_iterator.get_instances(data_type='valid')

x_test, y_test = train_iterator.get_instances(data_type='test')

print('train: ')
for x, y in list(zip(x_train, y_train))[:3]:
   print('x:', x)
   print('y:', y)
   print('=================')

print('validation: ')
for x, y in list(zip(x_valid, y_valid))[:3]:
   print('x:', x)
   print('y:', y)
   print('=================')

train: 
x: спасибо! у меня этот этюд купили еще до начала выставки (вот я удивилась и порадовалась!)
y: 0
x: Повысят с 270 на 280, из за ндс

y: 0
x: Вижу рожу киселя,сразу ставлю минус!

y: 0
validation: 
x: Пятый раз не у дел: Додона отстранили от власти
y: 1
x: Хуя, шамиль отца за член хватал, а хевин понавыдумывал.

y: 0
x: Страдай, изврощенец. Однако раньше рядом с хрюканом получше были тянки. А сейчас недотрапы колхозные и шизанутые.

y: 0


In [11]:
classifier_RoB.preparation(x_train, y_train, x_valid, y_valid )
#model.to(device)

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [12]:
"""from torch.utils.data import Dataset, DataLoader
from transformers import AdamW, get_linear_schedule_with_warmup


epochs = 6


train_set = CustomDataset(x_train, y_train, tokenizer)
valid_set = CustomDataset(x_valid, y_valid, tokenizer)

train_loader = DataLoader(train_set, batch_size=4, shuffle=True)
valid_loader = DataLoader(valid_set, batch_size=4, shuffle=True)

optimizer = AdamW(model.parameters(), lr=2e-5, correct_bias=False)
scheduler = get_linear_schedule_with_warmup(
         optimizer,
         num_warmup_steps=0,
         num_training_steps=len(train_loader) * epochs
         )
loss_fn = torch.nn.CrossEntropyLoss().to(device)"""

'from torch.utils.data import Dataset, DataLoader\nfrom transformers import AdamW, get_linear_schedule_with_warmup\n\n\nepochs = 6\n\n\ntrain_set = CustomDataset(x_train, y_train, tokenizer)\nvalid_set = CustomDataset(x_valid, y_valid, tokenizer)\n\ntrain_loader = DataLoader(train_set, batch_size=4, shuffle=True)\nvalid_loader = DataLoader(valid_set, batch_size=4, shuffle=True)\n\noptimizer = AdamW(model.parameters(), lr=2e-5, correct_bias=False)\nscheduler = get_linear_schedule_with_warmup(\n         optimizer,\n         num_warmup_steps=0,\n         num_training_steps=len(train_loader) * epochs\n         )\nloss_fn = torch.nn.CrossEntropyLoss().to(device)'

In [13]:
 torch.cuda.empty_cache()

In [14]:
all_valid_loss = []
all_train_loss = []
all_valid_acc = []
all_train_acc = []

"""model = model.train()
losses = []
correct_predictions = 0

for data in train_loader:
     print(data["text"])    
     input_ids = data["input_ids"].to(device)
     attention_mask = data["attention_mask"].to(device)
     targets = data["targets"].to(device)
     
     outputs = model(
          input_ids=input_ids,
          attention_mask=attention_mask,

      )
     
     preds = torch.argmax(outputs.pooler_output, dim=1)
     print(preds, targets.long())
     loss = loss_fn(outputs.pooler_output, targets)
     
     correct_predictions += torch.sum(preds == targets)

     losses.append(loss.item())
            
     loss.backward()
     torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
     optimizer.step()
     scheduler.step()
     optimizer.zero_grad()

train_acc = correct_predictions.double() / len(train_set)
train_loss = np.mean(losses)"""


all_valid_loss, all_valid_acc, all_train_loss, all_train_acc = classifier_RoB.train('RoBerta.pth')

Epoch 1/1
Train loss 4.672638054656982 accuracy 0.6552
Val loss 4.661413000106812 accuracy 0.671
----------


In [1]:
predictions = classifier_RoB.predict(x_test)
precision, recall, f1score = precision_recall_fscore_support(y_test, predictions)[:3]

print(f'precision: {precision}, recall: {recall}, f1score: {f1score}')

NameError: ignored

In [ ]:
import matplotlib.pyplot as plt 

fig, axs = plt.subplots(2, 2)
axs[0][0].plot([1,2,3,4], all_train_loss)
axs[0][1].plot([1,2,3,4], all_valid_loss)
axs[1][0].plot([1,2,3,4], all_train_acc)
axs[1][1].plot([1,2,3,4], all_valid_acc)